<a href="https://colab.research.google.com/github/arindam15964/mlrepo1/blob/master/mulcifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing modules
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
import cv2
import numpy as np
import numpy as np
from keras.preprocessing import image
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical
IMG_SIZE=128

Using TensorFlow backend.


In [0]:
#creating train data
import random
id_array=["0","1","2","3","4"]
#ls=[0,0,0,0,0]
train_data=[]
path2="/content/train"
def create_train_data():
  for category in id_array:
    newpath=os.path.join(path2,category)
    class_num=id_array.index(category)
    for img in os.listdir(newpath):
        img_array=cv2.imread(os.path.join(newpath,img),cv2.IMREAD_COLOR )
        new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        #plt.imshow(new_array,cmap="gray")
        #plt.show()
        #break
        if class_num==0 :
          i=2
        elif class_num==1 :
          i=4
        elif class_num==2 :
          i=1
        elif class_num==3 :
          i=2
        elif class_num==4 :
          i=2
        j=0
        while j<i:
          # ls[class_num]=ls[class_num]+1
          train_data.append([new_array,class_num])
          j=j+1
create_train_data()
random.shuffle(train_data)

In [4]:
X=[]
y=[]
i=0  
for features , labels in train_data:
  X.append(features)
  y.append(labels)
depth = 5
print(len(X))
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
X=X/255.0

3106


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42)

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

datagen.fit(X_train)


In [0]:
#creating the model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE,IMG_SIZE, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3),padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu'))
# model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 5, activation = 'softmax'))

In [0]:
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [0]:
z=to_categorical(y_train)
z1=to_categorical(y_test)

In [21]:
model.fit_generator(datagen.flow(X_train,z,
                                     batch_size=64),
                        epochs=50,
                        validation_data=(X_test,z1)
                        )

Epoch 1/50
44/44 [==============================] - 64s 1s/step - loss: 1.5197 - accuracy: 0.3059 - val_loss: 1.4796 - val_accuracy: 0.3569
Epoch 2/50
44/44 [==============================] - 62s 1s/step - loss: 1.4526 - accuracy: 0.3567 - val_loss: 1.4321 - val_accuracy: 0.3955
Epoch 3/50
44/44 [==============================] - 63s 1s/step - loss: 1.4342 - accuracy: 0.3667 - val_loss: 1.4203 - val_accuracy: 0.3794
Epoch 4/50
 3/44 [=>............................] - ETA: 1:08 - loss: 1.3690 - accuracy: 0.4062

KeyboardInterrupt: ignored

In [22]:
# Predicting USing the model
sdf=pd.read_csv("/content/sample.csv")
img_array=sdf['image']
pathf="/content/test/"
y_f=[]
for image1 in img_array:
  newpath=pathf+image1
  img=image.load_img(newpath,target_size=(IMG_SIZE,IMG_SIZE))
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  img=img/255
  # print(img)
  # break

  value=model.predict([img])
  y_f.append(value)
z_f=[]
for a in y_f:
  for data in a:
    z=np.argmax(data, axis=None)
    z_f.append(z)
    
  
dict={
    'image': img_array,
    'label': z_f
}
dff=pd.DataFrame.from_dict(dict)
dff.to_csv("ans.csv",index=False)
print(y_f)

[array([[0.06170643, 0.6132478 , 0.13776572, 0.16346438, 0.02381574]],
      dtype=float32), array([[0.07597284, 0.55235296, 0.13763747, 0.19760936, 0.03642737]],
      dtype=float32), array([[0.07845747, 0.5385495 , 0.16333   , 0.18551265, 0.03415047]],
      dtype=float32), array([[0.05965761, 0.602196  , 0.16055731, 0.16013752, 0.01745168]],
      dtype=float32), array([[0.07566649, 0.5105436 , 0.2044106 , 0.17612287, 0.0332565 ]],
      dtype=float32), array([[0.14120883, 0.3408583 , 0.25403726, 0.19206807, 0.07182752]],
      dtype=float32), array([[0.10749224, 0.46142486, 0.18736954, 0.19450887, 0.04920453]],
      dtype=float32), array([[0.07299437, 0.57848907, 0.14433192, 0.1767467 , 0.02743791]],
      dtype=float32), array([[0.09560744, 0.36758307, 0.28642485, 0.18414362, 0.06624101]],
      dtype=float32), array([[0.07916816, 0.5182167 , 0.18359509, 0.18329653, 0.03572342]],
      dtype=float32), array([[0.11643812, 0.4259886 , 0.20207755, 0.19575478, 0.0597409 ]],
      dty